In [2]:
import azureml.core
print('SDK version:', azureml.core.VERSION)

SDK version: 1.37.0


In [3]:
from azureml.core import Workspace

ws = Workspace(subscription_id="5aeeb535-21e1-4410-a3d0-539fa06445d7",
               resource_group="SignRecognition",
               workspace_name="SignRecognition")
ws

Workspace.create(name='SignRecognition', subscription_id='5aeeb535-21e1-4410-a3d0-539fa06445d7', resource_group='SignRecognition')

In [4]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='glamorous-wave',                  # Name of the registered model in your workspace.
                       model_path='../deploy-model/gw-model/model/glamorous-wave.onnx',              # Local ONNX model to upload and register as a model.
                       model_framework=Model.Framework.ONNX ,      # Framework used to create the model.
                       model_framework_version='1.3',              # Version of ONNX used to create the model.
                       description='Onnx Sign Recognition model')

print('Name:', model.name)

Registering model glamorous-wave
Name: glamorous-wave


In [5]:
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'onnx-sign-recognition-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-30 14:33:22+08:00 Creating Container Registry if not exists.
2021-12-30 14:33:22+08:00 Registering the environment.
2021-12-30 14:33:23+08:00 Use the existing image.
2021-12-30 14:33:24+08:00 Submitting deployment to compute.
2021-12-30 14:33:33+08:00 Checking the status of deployment onnx-sign-recognition-service..
2021-12-30 14:34:36+08:00 Checking the status of inference endpoint onnx-sign-recognition-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [8]:
import requests

headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}

if service.auth_enabled:
    headers['Authorization'] = 'Bearer '+ service.get_keys()[0]
elif service.token_auth_enabled:
    headers['Authorization'] = 'Bearer '+ service.get_token()[0]

scoring_uri = service.scoring_uri
print(scoring_uri)
with open('onnx-predict-input.json', 'rb') as data_file:
    response = requests.post(
        scoring_uri, data=data_file, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

http://ca92fac2-a3e7-4c0a-8c01-9715d61a6fd4.southeastasia.azurecontainer.io/score
400
0:00:00.037556
{'error_code': 400, 'error_message': 'Unexpected end of string. Expected a value.\n\n^'}
